In [1]:
import requests
import sys
import pandas as pd

sys.path.append('/home/sebastian/PycharmProjects/restructure/ProteinBoxBot_Core/')
import PBB_Core
import PBB_login




query = '''
SELECT * WHERE {
    {?cmpnd wdt:P279 wd:Q11173 .} UNION
    {?cmpnd wdt:P31 wd:Q11173 .} UNION
    {?cmpnd wdt:P662 [] .} UNION
    {?cmpnd wdt:P231 [] .} UNION
    {?cmpnd wdt:P235 [] .} UNION
    {?cmpnd wdt:P234 [] .}
    
    OPTIONAL {
        ?article schema:about ?cmpnd .
        ?article schema:inLanguage "en" .
        FILTER (SUBSTR(str(?article), 1, 25) = "https://en.wikipedia.org/") .
    }
  

    OPTIONAL{?cmpnd wdt:P2067 ?mass.} 
    OPTIONAL{?cmpnd wdt:P661 ?chemSpider .} 
    OPTIONAL{?cmpnd wdt:P662 ?pubchem_cid .} 
    OPTIONAL{?cmpnd wdt:P652 ?unii .} 
    OPTIONAL{?cmpnd wdt:P486 ?mesh_id .} 
    OPTIONAL{?cmpnd wdt:P665 ?kegg_id .} 
    OPTIONAL{?cmpnd wdt:P672 ?mesh_code .} 
    OPTIONAL{?cmpnd wdt:P683 ?chebi .} 
    OPTIONAL{?cmpnd wdt:P715 ?drugbank .} 
    OPTIONAL{?cmpnd wdt:P592 ?chembl .}
    OPTIONAL{?cmpnd wdt:P595 ?iuphar .}
    OPTIONAL{?cmpnd wdt:P231 ?cas .}
    OPTIONAL{?cmpnd wdt:P233 ?csmiles .}
    OPTIONAL{?cmpnd wdt:P234 ?inchi .}
    OPTIONAL{?cmpnd wdt:P235 ?inchi_key .}
    OPTIONAL{?cmpnd wdt:P274 ?chemical_formula .}
    OPTIONAL{?cmpnd wdt:P267 ?atc_code .}
    OPTIONAL{?cmpnd wdt:P2017 ?ismiles .}

}
ORDER BY ?cmpnd

'''

results = PBB_Core.WDItemEngine.execute_sparql_query(query)

In [2]:
print(len(results['results']['bindings']))

columns = results['head']['vars']
columns

105034


['cmpnd',
 'article',
 'mass',
 'chemSpider',
 'pubchem_cid',
 'unii',
 'mesh_id',
 'kegg_id',
 'mesh_code',
 'chebi',
 'drugbank',
 'chembl',
 'iuphar',
 'cas',
 'csmiles',
 'inchi',
 'inchi_key',
 'chemical_formula',
 'atc_code',
 'ismiles']

In [3]:
wd_chem_data = pd.DataFrame(columns=columns)
# dbpedia_chem_data.set_index

def get_all_single_compound_data(cmpnd):
    si = compound_index.index(cmpnd)
    rs = []
    for c, xx in enumerate(results['results']['bindings'][si:]):
        if compound_index[si + c] != cmpnd:
            break            
        rs.append(xx)

    for x in rs:
        yield {k: v['value'] for k, v in x.items() if k != 'cmpnd'}

count = 0
compound_index = [x['cmpnd']['value'] for x in results['results']['bindings']]

for cmpnd in compound_index:
    #print(cmpnd)
    if cmpnd in wd_chem_data['cmpnd'].values:
        continue
                       
    wd_chem_data.loc[count, 'cmpnd'] = cmpnd

    for zz in get_all_single_compound_data(cmpnd):
        for col, value in zz.items():
            if pd.notnull(wd_chem_data.loc[count, col]):
                td = set(wd_chem_data.loc[count, col].split('|'))
                td.add(value)
                wd_chem_data.loc[count, col] = '|'.join(td)
            else:
                wd_chem_data.loc[count, col] = value
                
    count += 1
    if count % 1000 == 0:
        print(count)
        #print(wd_chem_data[count-10:count])
        #break
            
wd_chem_data

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


,cmpnd,article,mass,chemSpider,pubchem_cid,unii,mesh_id,kegg_id,mesh_code,chebi,drugbank,chembl,iuphar,cas,csmiles,inchi,inchi_key,chemical_formula,atc_code,ismiles
0,http://www.wikidata.org/entity/Q153,https://en.wikipedia.org/wiki/Ethanol,46.1,682,702,3K9958V90M,M0000653,C00469,NaN,16236,00898,CHEMBL545,2299,64-17-5,CCO,"1S/C2H6O/c1-2-3/h3H,2H2,1H3",LFQSCWFLJHTTHZ-UHFFFAOYSA-N,C₂H₆O,V03AZ01|D08AX08|V03AB16,NaN
1,http://www.wikidata.org/entity/Q1002152,https://en.wikipedia.org/wiki/Fluoroiodomethane,NaN,10329326,13981373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,373-53-5,FCI,NaN,NaN,CH₂FI,NaN,NaN
2,http://www.wikidata.org/entity/Q1002155,https://en.wikipedia.org/wiki/Fluorosulfonate,NaN,2657785,3413884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[O-]S(F)(=O)=O,"1S/FHO3S/c1-5(2,3)4/h(H,2,3,4)/p-1",UQSQSQZYBQSBJZ-UHFFFAOYSA-M,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q1002165,https://en.wikipedia.org/wiki/Fluticasone,NaN,4470631,5311101,CUT2W21N7U,NaN,D07981,NaN,5134,NaN,CHEMBL1201396,NaN,90566-53-3,C[C@@H]1C[C@H]2[C@@H]3C[C@@H](C4=CC(=O)C=C[C@@...,1S/C22H27F3O4S/c1-11-6-13-14-8-16(24)15-7-12(2...,MGNNYOODZCAHBA-GQKYHHCASA-N,C₂₂H₂₇F₃O₄S,R01AD08|D07AC17|R03BA05,NaN
4,http://www.wikidata.org/entity/Q1003185,https://en.wikipedia.org/wiki/Fosfestrol,NaN,2297327,3032325,A0E0NMA80F,NaN,D00946,NaN,NaN,NaN,NaN,NaN,522-40-7,NaN,1S/C18H22O8P2/c1-3-17(13-5-9-15(10-6-13)25-27(...,NLORYLAYLIXTID-ISLYRVAYSA-N,C₁₈H₂₂O₈P₂,NaN,NaN
5,http://www.wikidata.org/entity/Q1003526,https://en.wikipedia.org/wiki/Bulbocapnine,NaN,11934,12441,O0TGI865QO,NaN,NaN,NaN,3211,NaN,CHEMBL157912,NaN,298-45-3,O1c4c(OC1)c3c2c(O)c(OC)ccc2C[C@H]5c3c(c4)CCN5C,1S/C19H19NO4/c1-20-6-5-11-8-14-19(24-9-23-14)1...,LODGIKWNLDQZBM-LBPRGKRZSA-N,C₁₉H₁₉NO₄,NaN,NaN
6,http://www.wikidata.org/entity/Q1007138,https://en.wikipedia.org/wiki/Gabaculine,NaN,3327,3445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87980-11-8,O=C(O)\C1=C\C=C/C(N)C1,"1S/C7H9NO2/c8-6-3-1-2-5(4-6)7(9)10/h1-3,6H,4,8...",KFNRJXCQEJIBER-UHFFFAOYSA-N,C₇H₉NO₂,NaN,NaN
7,http://www.wikidata.org/entity/Q1013950,https://en.wikipedia.org/wiki/Carbon%20monoxid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64972-88-9,NaN,NaN,NaN,NaN,NaN,NaN
8,http://www.wikidata.org/entity/Q1014238,https://en.wikipedia.org/wiki/Polythiazyl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56422-03-8,NaN,NaN,NaN,NaN,NaN,NaN
9,http://www.wikidata.org/entity/Q1014242,https://en.wikipedia.org/wiki/Ammonium%20hexaf...,NaN,11253835,24565,SW274S3321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7784-19-2,[NH4+].[NH4+].[NH4+].[F-].[F-].[F-].[F-].[F-]....,1S/Al.6FH.3H3N/h;6*1H;3*1H3/q+3;;;;;;;;;/p-3,OYHBNKHFKHBTRQ-UHFFFAOYSA-K,(NH₄)₃[AlF₆],NaN,NaN


In [4]:
wd_chem_data.head()


,cmpnd,article,mass,chemSpider,pubchem_cid,unii,mesh_id,kegg_id,mesh_code,chebi,drugbank,chembl,iuphar,cas,csmiles,inchi,inchi_key,chemical_formula,atc_code,ismiles
0,http://www.wikidata.org/entity/Q153,https://en.wikipedia.org/wiki/Ethanol,46.1,682,702,3K9958V90M,M0000653,C00469,NaN,16236,00898,CHEMBL545,2299,64-17-5,CCO,"1S/C2H6O/c1-2-3/h3H,2H2,1H3",LFQSCWFLJHTTHZ-UHFFFAOYSA-N,C₂H₆O,V03AZ01|D08AX08|V03AB16,NaN
1,http://www.wikidata.org/entity/Q1002152,https://en.wikipedia.org/wiki/Fluoroiodomethane,NaN,10329326,13981373,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,373-53-5,FCI,NaN,NaN,CH₂FI,NaN,NaN
2,http://www.wikidata.org/entity/Q1002155,https://en.wikipedia.org/wiki/Fluorosulfonate,NaN,2657785,3413884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[O-]S(F)(=O)=O,"1S/FHO3S/c1-5(2,3)4/h(H,2,3,4)/p-1",UQSQSQZYBQSBJZ-UHFFFAOYSA-M,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q1002165,https://en.wikipedia.org/wiki/Fluticasone,NaN,4470631,5311101,CUT2W21N7U,NaN,D07981,NaN,5134,NaN,CHEMBL1201396,NaN,90566-53-3,C[C@@H]1C[C@H]2[C@@H]3C[C@@H](C4=CC(=O)C=C[C@@...,1S/C22H27F3O4S/c1-11-6-13-14-8-16(24)15-7-12(2...,MGNNYOODZCAHBA-GQKYHHCASA-N,C₂₂H₂₇F₃O₄S,R01AD08|D07AC17|R03BA05,NaN
4,http://www.wikidata.org/entity/Q1003185,https://en.wikipedia.org/wiki/Fosfestrol,NaN,2297327,3032325,A0E0NMA80F,NaN,D00946,NaN,NaN,NaN,NaN,NaN,522-40-7,NaN,1S/C18H22O8P2/c1-3-17(13-5-9-15(10-6-13)25-27(...,NLORYLAYLIXTID-ISLYRVAYSA-N,C₁₈H₂₂O₈P₂,NaN,NaN


In [5]:
wd_chem_data.to_csv('wd_core_chem_data_concatenated-08-29-2016.csv')

In [6]:
wd_chem_data.count()

cmpnd               24810
article             16391
mass                  637
chemSpider          11463
pubchem_cid         16909
unii                11826
mesh_id               927
kegg_id              4065
mesh_code               3
chebi                4464
drugbank             2682
chembl               5461
iuphar               1033
cas                 19692
csmiles             15636
inchi               14471
inchi_key           14946
chemical_formula    19475
atc_code             1709
ismiles                16
dtype: int64

In [8]:
import pprint
import urllib
import requests
import pandas as pd

chem_parameters = [
    'title',
    'SystematicName',
    'OtherNames',
    'synonyms',
    'IUPHAR_ligand',
    'CASNo',
    'CAS_number',
    'UNII',
    'SMILES',
    'KEGG',
    'ChEMBL',
    'ChemSpiderID',
    'DrugBank',
    'PubChem',
    'StdInChI',
    'StdInChIKey',
    'InChI',
    'InChIKey',
    'ATCCode_prefix',
    'ATC_prefix',
    'ATCCode_suffix',
    'ATC_suffix',
    'ATC_Supplemental',
    'molecular_weight',
    'page_title'
]


def parse_chembox(pages):
    # querying one page at a time is most conventient to code

    url = 'https://en.wikipedia.org/w/api.php'

    if not pages:
        pages = ['Imatinib', 'Vemurafenib']

    params = {
            'action': 'query',
            'titles': '|'.join(pages),
            'prop': 'revisions',
            'rvprop': 'content',
            'format': 'json'
        }
    r = requests.get(url=url, params=params).json()
    # pprint.pprint(r)
    for zz in [x for x in r['query']['pages'].values()]:
        current_page_title = zz['title']
        page_text = zz['revisions'][0]['*']
        # pprint.pprint(page_text)
        # print(page_text)

        for line in page_text.split('\n'):
            if line.strip().startswith('|'):
                splits = line.strip().split(' ')
                if len(splits) < 2:
                    continue
                chem_param = splits[1]
                
                if splits[-1] == '=':
                    chem_param = chem_param[:-1]

                value = line.split('=', maxsplit=1)[-1].strip()

                if chem_param in chem_parameters:
                    # print(chem_param, value)
                    yield {chem_param: value}
#             elif line.strip().startswith("'''"):
#                 name = line.split("'''")[1]
#                 yield {'title': name}

        yield {'page_title': current_page_title}
        

s = requests.Session()
def infobox_backlinks(ib):
    base_url = 'https://en.wikipedia.org/w/api.php'
    for i in ib:

        params = {
            'action': 'query',
            'list': 'embeddedin',
            'eititle': i,
            'eilimit': 500,
            'format': 'json',
            'eifilterredir': 'nonredirects',
            'einamespace': 0
        }

        r = s.get(url=base_url, params=params)
        backlinks = r.json()['query']['embeddedin']

        if 'continue' in r.json():
            cont = r.json()['continue']
            params.update(cont)
#             pprint.pprint(params)
            while True:
                r = s.get(url=base_url, params=params)
                if 'embeddedin' in r.json()['query']:
                    backlinks.extend(r.json()['query']['embeddedin'])

                if 'continue' in r.json():
                    cont = r.json()['continue']
                    params.update(cont)
                else:
                    break

        for z in backlinks:
            yield z
  

infoboxes = ['Template:Infobox_drug', 'Template:Chembox']

wp_chem_data = pd.DataFrame(columns=chem_parameters)
for count, art in enumerate(infobox_backlinks(infoboxes)):
    title = art['title']
    print(count, title)
#     pprint.pprint(list(parse_chembox([page_title])))
    
    all_data = list(parse_chembox([title]))
    for x in :
        print(x)
#         'CAS_number'

    if count > 300:
        break
        

0 Aspirin
{'IUPHAR_ligand': '4139'}
{'CAS_number': '50-78-2'}
{'ATC_prefix': 'A01'}
{'ATC_suffix': 'AD05'}
{'PubChem': '2244'}
{'DrugBank': 'DB00945'}
{'ChemSpiderID': '2157'}
{'UNII': 'R16CO5Y76E'}
{'KEGG': 'D00109'}
{'ChEMBL': '25'}
{'molecular_weight': '180.157 g/mol'}
{'StdInChI': '1S/C9H8O4/c1-6(10)13-8-5-3-2-4-7(8)9(11)12/h2-5H,1H3,(H,11,12)'}
{'StdInChIKey': 'BSYNRYMUTXBXSQ-UHFFFAOYSA-N'}
{'synonyms': '2-acetoxybenzoic acid<br />acetylsalicylate<br />acetylsalicylic acid<br />O-acetylsalicylic acid'}
{'page_title': 'Aspirin'}
1 Ampicillin
{'CAS_number': '69-53-4'}
{'ATC_prefix': 'J01'}
{'ATC_suffix': 'CA01'}
{'PubChem': '6249'}
{'DrugBank': 'DB00415'}
{'ChemSpiderID': '6013'}
{'UNII': '7C782967RD'}
{'KEGG': 'D00204'}
{'ChEMBL': '174'}
{'molecular_weight': '349.41 g/mol'}
{'SMILES': 'O=C(O)[C@@H]2N3C(=O)[C@@H](NC(=O)[C@@H](c1ccccc1)N)[C@H]3SC2(C)C'}
{'StdInChI': '1S/C16H19N3O4S/c1-16(2)11(15(22)23)19-13(21)10(14(19)24-16)18-12(20)9(17)8-6-4-3-5-7-8/h3-7,9-11,14H,17H2,1-2H3,(H,18,